This is the final project with all of the experiments stripped out.

### load modules

In [1]:
import sys
import pickle
sys.path.append("../tools/")

import pandas as pd
import numpy as np

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import Imputer
                                                
from pprint import pprint
from sklearn.tree import DecisionTreeClassifier

### import data into Pandas DataFrame

In [2]:
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)
df = pd.DataFrame(data_dict)
df = df.transpose()
df.head()

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
ALLEN PHILLIP K,4175000,2869717,-3081055,NaN,phillip.allen@enron.com,1729541,13868,2195,47,65,...,304805,152,False,126027,-126027,201955,1407,2902,4484442,1729541
BADUM JAMES P,NaN,178980,NaN,NaN,NaN,257817,3486,NaN,NaN,NaN,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,182466,257817
BANNANTINE JAMES M,NaN,NaN,-5104,NaN,james.bannantine@enron.com,4046157,56301,29,39,0,...,NaN,864523,False,1757552,-560222,477,465,566,916197,5243487
BAXTER JOHN C,1200000,1295738,-1386055,NaN,NaN,6680544,11200,NaN,NaN,NaN,...,1586055,2660303,False,3942714,NaN,267102,NaN,NaN,5634343,10623258
BAY FRANKLIN R,400000,260455,-201641,NaN,frank.bay@enron.com,NaN,129142,NaN,NaN,NaN,...,NaN,69,False,145796,-82782,239671,NaN,NaN,827696,63014


### Task 1: Select Features

In [3]:
df = df.drop('email_address', axis=1)
df = df.astype(float)

### Task 2: Eliminate Outliers/Missing Data

In [4]:
df = df.drop('TOTAL')
df = df.drop("THE TRAVEL AGENCY IN THE PARK")
df = df.drop("loan_advances", axis=1)
#letting the imputer deal with missing data
#df.fillna(0, inplace=True)

### Task 3: Create New Features

In [5]:
df['deferred_ratio'] = df['deferred_income']/(df['total_payments'] + 1)

#### sklearn prep: features_list

In [6]:
features_list = list(df.columns)
features_list.remove('poi')

In [7]:
len(features_list)

19

### Task 4: Try a variety of classifiers

First I am going to try the GridSearchCV tutorial with the data in the machine learning class. 

From page 1, I start with the data in a pandas data frame. 

We don't have any categorical variables besides the dependent variable so I will not have to use the sklearn.preprocessing.LabelEncoder() or the pandas.get_dummies() functions. 

The data is in a pandas data frame and has not been split into training or testing data sets. So what we have to do is create the features and labels. 

In [8]:
labels = df['poi']

In [9]:
features = df[features_list]

In [10]:
from sklearn import cross_validation
features_train, features_test, labels_train, labels_test = \
    cross_validation.train_test_split(features, labels, test_size=0.3, random_state=42)

In [11]:
len(features_train.columns)

19

In [12]:
len(features_test.columns)

19

In [13]:
print "length of training: %r\nlength of test: %r" % (len(features_train), len(features_test))

length of training: 100
length of test: 44


In [14]:
features_test.describe()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,deferred_ratio
count,25.000000,14.000000,11.000000,3.000000,29.000000,29.000000,30.000000,30.000000,30.000000,19.000000,29.000000,35.000000,6.000000,29.000000,30.000000,30.000000,38.000000,38.000000,11.000000
mean,1264800.000000,662323.285714,-522176.909091,98500.666667,3126352.275862,49334.000000,638.433333,61.966667,60.933333,556129.000000,232793.448276,748024.314286,2490040.333333,247592.103448,1199.400000,2363.300000,2014551.631579,3061289.973684,-1.891790
std,1700498.382828,1031184.388916,885882.506154,52916.611166,5580156.736252,39144.547234,1417.034462,106.853262,143.676824,542692.247151,420959.519439,1087385.799032,6352191.596559,80992.065895,1225.187715,3482.927902,2935916.274954,5082618.262049,5.288708
min,70000.000000,10259.000000,-3081055.000000,38346.000000,59539.000000,475.000000,12.000000,0.000000,0.000000,69223.000000,2.000000,-2604490.000000,-140264.000000,76399.000000,10.000000,136.000000,475.000000,63014.000000,-17.818773
25%,400000.000000,51930.000000,-590337.500000,78819.000000,1030329.000000,16514.000000,27.500000,10.000000,0.250000,250000.000000,1231.000000,322088.500000,-118159.250000,211844.000000,219.500000,535.500000,450568.750000,698411.500000,-0.597028
50%,750000.000000,201073.500000,-200000.000000,119292.000000,1635238.000000,38559.000000,40.500000,26.000000,6.500000,369721.000000,2401.000000,441096.000000,-88669.000000,249201.000000,830.500000,1324.000000,1185341.000000,1786678.500000,-0.243617
75%,1100000.000000,743196.750000,-87500.000000,128578.000000,2604490.000000,77978.000000,202.250000,46.250000,45.250000,547586.500000,400572.000000,885438.500000,-75009.750000,288542.000000,1554.750000,2572.750000,2073245.250000,3571813.250000,-0.093823
max,8000000.000000,3131860.000000,-4167.000000,137864.000000,30766064.000000,137767.000000,6759.000000,528.000000,609.000000,2035380.000000,1573324.000000,4131594.000000,15456290.000000,404338.000000,4527.000000,15149.000000,15456290.000000,30766064.000000,-0.002079


The first model I will try this with is decision trees since that is what they go through in the tutorial. 

In [15]:
imp = Imputer(missing_values='NaN', strategy='median')
features_train_imp = imp.fit_transform(features_train)
features_test_imp = imp.fit_transform(features_test)

In [16]:
len(features_train_imp[0][:])

19

In [17]:
len(features_test_imp[0][:])

19

In [18]:
clf = DecisionTreeClassifier(random_state = 53)
clf = clf.fit(features_train_imp, labels_train)

In [19]:
len(features_test_imp[0][:])

19

In [20]:
labels_pred = clf.predict(features_test_imp)
print "Recall score:"
print recall_score(labels_test, labels_pred)
print "\n"

print "Precision score:"
print precision_score(labels_test, labels_pred)
print "\n"

print "Decision Tree Accuracy score:"
print clf.score(features_test_imp, labels_test)

print "\nDecision Tree Parameters:"
pprint(clf.get_params())


# precision_recall_fscore_support(labels_test, labels_pred)a

Recall score:
0.0


Precision score:
0.0


Decision Tree Accuracy score:
0.75

Decision Tree Parameters:
{'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': 53,
 'splitter': 'best'}


I can't believe that we get 0 scores for both precision and recall, so I investigate further. 

In [21]:
count = 0
for i in range(len(labels_test)):
    if labels_test[i] == 1 and labels_pred[i]:
        count += 1
        print "got one!"
print count

0


And I still can't believe we get nothing so I try one more test. I print out the test set labels and the predicted values for the labels side by side and visually inspect them. Finally, I am satisfied that the decision tree model gets us absolutely nothing. Its high accuracy score is merely an artifact of the very low number of positives making it possible for a model to achieve fairly high accuracy by simply predicting 0 for person of interest most of the time. 

In [22]:
for i in range(len(labels_test)):
    print "True: %r Pred: %r\n" %(labels_test[i], labels_pred[i])

True: 0.0 Pred: 1.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 1.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 1.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 1.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 1.0

True: 0.0 Pred: 0.0

True: 1.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 1.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 1.0

True: 1.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 1.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 0.0

True: 0.0 Pred: 1.0

True: 0.0 Pred: 0.0



So, unless I find some other way to do make a bunch of train-test splits when I run a model I may not be able to get anywhere just running looking for models with high recall on just the single small data set we have here. I am going to try the grid search with the scoring function set to recall but we are going to have to have to think that the recall score might not give us enough to work with. 

The cross validation parameter of the grid search may save the day. It may, though I am not sure, make some changes in the train-test split somehow, sampling from within the data set, to give us more leverage. I will have to try it out to see. 

### Grid Search Cross Validation
So we make lists of the parameters of the model that we want to search through. Then there are parameters of the GridSearchCV model itself. Then we fit the model, test its predictions and print out the best estimator and its parameters. 

In [41]:
from sklearn import grid_search
#create a dictionary with the parameeters we want to search through
param_grid = {'min_samples_split': [1, 5, 10, 15, 20, 25, 30],
             'max_depth': [4, 5, 6, 7, 8]}

#create the grid_search object
gridCV_object = grid_search.GridSearchCV(estimator = DecisionTreeClassifier(random_state=53),
                                        param_grid = param_grid,
                                        cv = 5)

#fit to the data
gridCV_object.fit(features_train_imp, labels_train)

#find the best parameters chosen
print "Best parameters from the parameter grid:" 
pprint(gridCV_object.best_params_)

#get the best estimator
clf_2 = gridCV_object.best_estimator_

#get the best parameters from the best estimator
print "\nComplete set of parameters for best estimator:"
pprint(clf_2.get_params())

#compare scores for the default and the grid search decision trees
print "\nAccuracy score for default decision tree:"
print clf.score(features_test_imp, labels_test)

print "\nAccuracy score for the grid search estimator:"
print clf_2.score(features_test_imp, labels_test)

labels_clf_pred = clf.predict(features_test_imp)

labels_clf2_pred = clf_2.predict(features_test_imp)

print "\nF1 score for default decision tree: %r\nF1 score for grid search: %r" %(recall_score(labels_test, labels_clf_pred), recall_score(labels_test, labels_clf2_pred) )

Best parameters from the parameter grid:
{'max_depth': 4, 'min_samples_split': 25}

Complete set of parameters for best estimator:
{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 4,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 25,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': 53,
 'splitter': 'best'}

Accuracy score for default decision tree:
0.75

Accuracy score for the grid search estimator:
0.772727272727

F1 score for default decision tree: 0.0
F1 score for grid search: 0.20000000000000001


So, there first time there was a big drop for the grid search estimator. Now, with the loan_advances variable dropped from the beginning, the accuracy score goes up with the grid search. Now the big problem is to figure out how to get the grid search optimize on the recall variable rather than the accuracy score.  

In [42]:
from sklearn import metrics
labels_pred_2 = clf_2.predict(features_test_imp)
print recall_score(labels_test, labels_pred_2) 
print precision_score(labels_test, labels_pred_2)

0.2
0.142857142857


Ok, so the decision tree classifier is absolutely terrible but it is at least doing something after grid search as opposed to the tree classifier with just the default parameters which was just flat out zero. 

#### Tweeking the Decision Tree: StratifiedShuffleSplit
This is a model that, I believe, lets you get a bit more milliage out of the grid search by doing splits of the training data to get more robust tests of the data making up for the small size of the data set. 

I am going to try to find the way to do this with the recall score guiding the seleciton of the model. 

In [43]:
#keep same param_grid for now
#create StratefiedShuffleSplit object
from sklearn.cross_validation import StratifiedShuffleSplit
cross_validator = StratifiedShuffleSplit(y = labels_train, random_state = 0)

#create grid_search object
gridCV_object = grid_search.GridSearchCV(estimator = DecisionTreeClassifier(random_state=53),
                                        param_grid = param_grid,
                                        scoring = 'recall',
                                        cv = cross_validator)

#fit to the data to the new grid search
gridCV_object.fit(features_train_imp, labels_train)

#what were the parameteres chosen from the grid?
print "Parameters chosen from the grid:"
pprint(gridCV_object.best_params_)

#get best estimator
print "Best estimator:"
clf_recall = gridCV_object.best_estimator_

#get the best parameters for the best estimator
print "The best parameters for the best estimator:"
pprint(clf_recall.get_params())

#check the scores
from sklearn.metrics import recall_score
print "\n Recall score for default decision tree:"
predictions_recall = clf.predict(features_test_imp)
print recall_score(predictions_recall, labels_test)

#now for the grid search determined parameters:
print "\nRecall score for best estimator:"
predictions_recall = clf_recall.predict(features_test_imp)
print recall_score(predictions_recall, labels_test)

Parameters chosen from the grid:
{'max_depth': 4, 'min_samples_split': 15}
Best estimator:
The best parameters for the best estimator:
{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 4,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 15,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': 53,
 'splitter': 'best'}

 Recall score for default decision tree:
0.0

Recall score for best estimator:
0.142857142857


#### Recall
Now I am going to do the whole thing for the recall score, still using the decision tree but using a wider set of parameters and using the stratified shuffle split testing method. 

In [44]:
#keep same param_grid for now
#create StratefiedShuffleSplit object
from sklearn.cross_validation import StratifiedShuffleSplit
cross_validator = StratifiedShuffleSplit(y = labels_train, random_state = 0)

param_grid = {'min_samples_split': [1, 2, 3, 4, 5, 6, 10, 15, 20, 25, 30],
             'max_depth': [4, 5, 6, 7, 8]}

#create grid_search object
gridCV_object = grid_search.GridSearchCV(estimator = DecisionTreeClassifier(random_state=53),
                                        param_grid = param_grid,
                                        scoring = 'recall',
                                        cv = cross_validator)

#fit to the data to the new grid search
gridCV_object.fit(features_train_imp, labels_train)

#what were the parameteres chosen from the grid?
print "Parameters chosen from the grid:"
pprint(gridCV_object.best_params_)

#get best estimator
print "Best estimator:"
clf_recall = gridCV_object.best_estimator_

#get the best parameters for the best estimator
print "The best parameters for the best estimator:"
pprint(clf_recall.get_params())

#check the scores
from sklearn.metrics import recall_score
print "\nRecall score for default decision tree:"
predictions_1 = clf.predict(features_test_imp)
print recall_score(predictions_1, labels_test)

#now for the grid search determined parameters:
print "\nRecall score for best estimator:"
predictions_recall = clf_recall.predict(features_test_imp)
print recall_score(predictions_recall, labels_test)

Parameters chosen from the grid:
{'max_depth': 4, 'min_samples_split': 15}
Best estimator:
The best parameters for the best estimator:
{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 4,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 15,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': 53,
 'splitter': 'best'}

Recall score for default decision tree:
0.0

Recall score for best estimator:
0.142857142857


### Pipelines
The next step is to use all of this wonderful grid searching capability on the Gaussian Naive Bayes model. That will get me the best parameters.

First, I look up what the possible parameters are for the GaussianNB. Of course, this will also require that I normalize the data and do the Principle Components analysis on the data before it goes to the model so I will also have to do some data preparation and build a pipeline. 

The Gaussian Naive Bayes model takes the parameters of--ok, well it doesn't really have any parameters to take. It just wants the normalized data and the principle components if you are doing that. So, we will skip the Gaussian for now and, following the tutorial, add in the pipeline for the decision tree classifier. 

In [50]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# imp = Imputer(missing_values='NaN', strategy='0')
# features_train = imp.fit_transform(features_train)
# features_test = imp.fit_transform(features_test)

Pipeline = Pipeline([
        ('imp', Imputer(missing_values='NaN', strategy='most_frequent')),
        ('std', StandardScaler()),
        ('pca', PCA()),
        ('clf', DecisionTreeClassifier(random_state = 53))
    ])

pipeline = Pipeline.fit(features_train, labels_train)

print "Pipeline Accuracy Score:"
print pipeline.score(features_test, labels_test)

print "\nPipeline parameters:"
pprint(pipeline.get_params())

clf_pipeline_pred = pipeline.predict(features_test)
print "\n\nRecall Score:", recall_score(labels_test, clf_pipeline_pred)

Pipeline Accuracy Score:
0.840909090909

Pipeline parameters:
{'clf': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=53, splitter='best'),
 'clf__class_weight': None,
 'clf__criterion': 'gini',
 'clf__max_depth': None,
 'clf__max_features': None,
 'clf__max_leaf_nodes': None,
 'clf__min_samples_leaf': 1,
 'clf__min_samples_split': 2,
 'clf__min_weight_fraction_leaf': 0.0,
 'clf__presort': False,
 'clf__random_state': 53,
 'clf__splitter': 'best',
 'imp': Imputer(axis=0, copy=True, missing_values='NaN', strategy='most_frequent',
    verbose=0),
 'imp__axis': 0,
 'imp__copy': True,
 'imp__missing_values': 'NaN',
 'imp__strategy': 'most_frequent',
 'imp__verbose': 0,
 'pca': PCA(copy=True, n_components=None, whiten=False),
 'pca__copy': True,
 'pca__n_components': None,
 'pca__whiten': Fa

Now we are going to run the grid search with the pipeline put in as the estimator. The things we want to change in the pipeline will be put in through the grid search. 

In [52]:
param_grid = {'imp__strategy': ['median', 'most_frequent'], 
              'clf__min_samples_split': [1, 2, 3, 4, 5, 6, 10, 15, 20, 25, 30],
              'clf__max_depth': [4, 5, 6, 7, 8],
              'pca__n_components': [2,3,4,5,6,7]}

#create the GridSearchCV object using param_grid
gridCV_object = grid_search.GridSearchCV(estimator = pipeline, 
                                        param_grid = param_grid,
                                        cv = 8)

gridCV_object.fit(features_train, labels_train)

GridSearchCV(cv=8, error_score='raise',
       estimator=Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='most_frequent',
    verbose=0)), ('std', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, n_components=None, whiten=False)), ('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', ...plit=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=53, splitter='best'))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'pca__n_components': [2, 3, 4, 5, 6, 7], 'clf__max_depth': [4, 5, 6, 7, 8], 'clf__min_samples_split': [1, 2, 3, 4, 5, 6, 10, 15, 20, 25, 30], 'imp__strategy': ['median', 'most_frequent']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [53]:
print "Best parameters from the grid search:", pprint(gridCV_object.best_params_)

Best parameters from the grid search:{'clf__max_depth': 4,
 'clf__min_samples_split': 10,
 'imp__strategy': 'median',
 'pca__n_components': 7}
 None


In [54]:
clf_gridCV = gridCV_object.best_estimator_

In [55]:
print "\nBest estimator accuracy:", clf_gridCV.score(features_test, labels_test)


Best estimator accuracy: 0.863636363636


In [56]:
clf_gridCV_pred = clf_gridCV.predict(features_test)
print "\n\nRecall Score:", recall_score(labels_test, clf_gridCV_pred)



Recall Score: 0.0


Why didn't the recall score go up? What did I do wrong? 

Now we can switch out the tree classifier for the random forest classifier

In [63]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
        ('imp', Imputer()),
        ('std', StandardScaler()),
        ('pca', PCA()),
        ('clf', RandomForestClassifier(random_state=53, class_weight = 'balanced'))
    ])

param_grid = {'imp__strategy':['median', 'most_frequent'],
              'clf__max_depth': [4,5,6,7,8],
              'pca__n_components': [2,3,4,5,6,7]}

gridCV_object = grid_search.GridSearchCV(estimator = pipeline, 
                                        param_grid = param_grid,
                                        cv = 8)

gridCV_object.fit(features_train, labels_train)

GridSearchCV(cv=8, error_score='raise',
       estimator=Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('std', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, n_components=None, whiten=False)), ('clf', RandomForestClassifier(bootstrap=True, class_weight='balanced',
         ...stimators=10, n_jobs=1,
            oob_score=False, random_state=53, verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'pca__n_components': [2, 3, 4, 5, 6, 7], 'clf__max_depth': [4, 5, 6, 7, 8], 'imp__strategy': ['median', 'most_frequent']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

Now we look at the preformance of this model. Now that we have fit the model we look at how well it does. First we print out the parameters of the model that was chosen by grid search as the best one. 

In [64]:
print "Best parameters from the grid search:", pprint(gridCV_object.best_params_)

Best parameters from the grid search:{'clf__max_depth': 6, 'imp__strategy': 'median', 'pca__n_components': 2}
 None


Ok, now we have the model that is best. It goes to a depth of 6 in the decision tree, it uses the median to replace missing values and it uses two principle components. 

Now, we look at how well it does in recall, precision and the f1 score. To do that we have to make that best model into an object, use that object to make predictions, and then use those predictions to get the scores of the model on the recall, precision and f1 measures. 

Make the model an object with a name based on what it is, a classifier using random forest and grid search, so clf_rf_gridCV. 

In [65]:
clf_rf_gridCV = gridCV_object.best_estimator_

Now we have to use that model to get predictions

In [67]:
y_rf_pred = clf_rf_gridCV.predict(features_test)

Now that we have the predictions we use them to get recall score.

In [69]:
print "\n\nRecall Score:", recall_score(labels_test, y_rf_pred)

 

Recall Score: 0.0


In [45]:
#keep same param_grid for now
#create StratefiedShuffleSplit object
from sklearn.cross_validation import StratifiedShuffleSplit
cross_validator = StratifiedShuffleSplit(y = labels_train, random_state = 0)

param_grid = {'min_samples_split': [1, 2, 3, 4, 5, 6, 10, 15, 20, 25, 30],
             'max_depth': [4, 5, 6, 7, 8]}

#create grid_search object
gridCV_object = grid_search.GridSearchCV(estimator = DecisionTreeClassifier(random_state=53),
                                        param_grid = param_grid,
                                        scoring = 'recall',
                                        cv = cross_validator)

#fit to the data to the new grid search
gridCV_object.fit(features_train_imp, labels_train)

#what were the parameteres chosen from the grid?
print "Parameters chosen from the grid:"
pprint(gridCV_object.best_params_)

#get best estimator
print "Best estimator:"
clf_recall = gridCV_object.best_estimator_

#get the best parameters for the best estimator
print "The best parameters for the best estimator:"
pprint(clf_recall.get_params())

#check the scores
from sklearn.metrics import recall_score
print "\nRecall score for default decision tree:"
predictions_1 = clf.predict(features_test_imp)
print recall_score(predictions_1, labels_test)

#now for the grid search determined parameters:
print "\nRecall score for best estimator:"
predictions_recall = clf_recall.predict(features_test_imp)
print recall_score(predictions_recall, labels_test)

Parameters chosen from the grid:
{'max_depth': 4, 'min_samples_split': 15}
Best estimator:
The best parameters for the best estimator:
{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 4,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 15,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': 53,
 'splitter': 'best'}

Recall score for default decision tree:
0.0

Recall score for best estimator:
0.142857142857


### Task 5: Tune precision and recall > 0.3

#### Standardize by subtracting mean and dividing by std_dev

In [11]:
from sklearn import preprocessing
# Create a minimum and maximum processor object
standard_scaler = preprocessing.StandardScaler()

# Create an object to transform the data to fit minmax processor
features_train = standard_scaler.fit_transform(features_train)
features_test = standard_scaler.fit_transform(features_test)

Now I am going to use the select k best method on the data and then see how it does with the models. 

COACH QUESTION: Where do I do the k-best thing? Can it be added to a pipeline below? 

In [12]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
test_classifier(clf, my_dataset, features_list, folds = 1000)

GaussianNB()
	Accuracy: 0.61633	Precision: 0.21514	Recall: 0.70900	F1: 0.33011	F2: 0.48592
	Total predictions: 15000	True positives: 1418	False positives: 5173	False negatives:  582	True negatives: 7827



COACH QUESTION: Why doesn't the rbf kernel work? 

In [13]:
from sklearn import svm
clf = svm.SVC(kernel='rbf',C=100) 
# tried gamma=0.001, C=100.
# kernel = 'linear', kernel='rbf'
test_classifier(clf, my_dataset, features_list, folds=1000)

Got a divide by zero when trying out: SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Precision or recall may be undefined due to a lack of true positive predicitons.


In [14]:
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.decomposition import PCA
estimators = [('reduce_dim', PCA(n_components=7)), ('svm', svm.SVC(kernel='rbf'))]
clf = Pipeline(estimators)
test_classifier(clf, my_dataset, features_list, folds = 1000)

Got a divide by zero when trying out: Pipeline(steps=[('reduce_dim', PCA(copy=True, n_components=7, whiten=False)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
Precision or recall may be undefined due to a lack of true positive predicitons.


### The Winner
COACH QUESTION: So the Guassian Naive Bayes with the dimensions of the data set reduced to 8 through prinicple component analysis is the one that I got to break the 0.3 thresholds in Recall and Precision. I did it by just doing a manual search through the possible number of components. I have no idea which components were chosen and on what basis they worked, which 8 worked better than 9 or why 7 didn't make the 0.3 cut at all. I have no intuition for why any of this worked. And I don't know how to visualize it. Can I just submit this for the final project? 

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
estimators = [('reduce_dim', PCA(n_components=8)), ('Gaussian', GaussianNB())]
clf = Pipeline(estimators)
test_classifier(clf, my_dataset, features_list, folds = 1000)

Pipeline(steps=[('reduce_dim', PCA(copy=True, n_components=8, whiten=False)), ('Gaussian', GaussianNB())])
	Accuracy: 0.85053	Precision: 0.42568	Recall: 0.34650	F1: 0.38203	F2: 0.35989
	Total predictions: 15000	True positives:  693	False positives:  935	False negatives: 1307	True negatives: 12065

